# Learning the parameters of a MR-Sort model using a readapted version of an existing metaheuristic algorithm (oso-pymcda) in context of unknown preference directions

## Introduction

We intend in this document to brievely describe our version of a metaheuristic algorithm for the learning of MR-Sort model parameters without knowing in advance the preference directions of some criteria.

The initial metaheuristic algorithm at our disposal comes from github (https://github.com/oso/pymcda) developped by Olivier Sobrie). This algorithm is the starting point on which we present our approach.

More specifically, this proposition - that can be seen as an extension of the existing metaheuristic - integrate more parameters, in particular preference directions on each criteria. We also include the computation preference direction restoration rate accuracy, as well as the generation of statistic plots on the learning results. 

The code originally in Python 2 has been upgraded to some extent to Python 3 and is located in the "oso-pymcda/" directory, which is in the same directory as this notebook.

## Some helpful articles on MRSort models

In order to grasp the metaheuristic that is used in this notebook and have a overview on the methodology of MCDA, we give here below some useful related articles :
   * [Learning monotone preferences using a majority rule sorting model](papers/Sobrie_Mousseau_Pirlot.pdf) (in particular, this explains with details the procedure of the metaheuristic)
   * [Learning the Parameters of a Multiple Criteria Sorting Method Based on a Majority Rule](papers/Leroy_Mousseau_Pirlot.pdf)
   * [A new decision support model for preanesthetic evaluation](papers/Sobrie_and_al.pdf)


## Settings

Before digging into the code, here are some requirements to have : 


* The version of Python used for this notebook is 3.7. Please check if you have the right version with this command on a terminal : *python --version* . If not, you can download this version on https://www.python.org/downloads/.

 * You may need to download Anaconda3 (you will find here the complete procedure : https://docs.anaconda.com/anaconda/install/mac-os/)

* The library matplotlib.pyplot need to be installed. This can be done with the command line below (preferably using pip  - that can be also installed following the instructions of this link : https://pip.pypa.io/en/stable/installing/):

In [1]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


   * Download CPLEX Optimization Studio. Go to https://www.ibm.com/products/ilog-cplex-optimization-studio (choose the student/teacher free edition)  and follow the steps until the download of the "ILOG CPLEX Optimization Studio" following your operating system. The CPLEX version used in this notebook is 12.9. You may have to create a IBMid account. 

   * Then, look at instructions in the ReadMe file of the CPLEX directory that has been created in the Applications directory. In particular, it may require that you update your Java runtime application.

   * Open also the ReadMe file in the python directory of the CPLEX directory. Execute this command line on the terminal : *pip install docplex*

   * In order to set up the CPLEX Python API, follow instructions here : https://www.ibm.com/support/knowledgecenter/SSSA5P_12.9.0/ilog.odms.cplex.help/CPLEX/GettingStarted/topics/set_up/Python_setup.html. In the same directory as previously, execute the command line on the terminal  : *python setup.py install*

  * Set the environment variable PYTHONPATH on the terminal so that it may contains the path from the root folder to "cplex" via "Anaconda3" and another path from the "Applications" folder to "cplex". Here is an example : *export PYTHONPATH=$PYTHONPATH:/Users/pegdwendeminoungou/anaconda3/lib/python3.7/site-packages/cplex:/Applications/CPLEX_Studio129/cplex*


* Any help could be found here : https://www.ibm.com/support/knowledgecenter/SSSA5P_12.9.0/ilog.odms.studio.help/Optimization_Studio/topics/COS_home.html
   

We need to set the global variable *DATADIR* so that it contains the right path from the root to this working directory  **MRSort-jupyter-notebook** . Here an example :

In [2]:
%env DATADIR /Users/pegdwendeminoungou/python_workspace/MRSort-jupyter-notebook

env: DATADIR=/Users/pegdwendeminoungou/python_workspace/MRSort-jupyter-notebook


## Brief description of the metaheuristic for learning preference directions

As stated before, our approach is based on a evolutionrary algorithm.
It consist in the generation and evolution of heterogeneous models (models with both increasing and decreasing preference directions on some criteria) in the population.
The goal of this readaptation is to foster the evolution of good models, those on which criteria possess the true preference directions.

The implementation relies on a 3 axes : the mechanism of generation and renewal of the population of model, the core strategy of the method acting on model weights and profiles, and finally the decision rule on the selection of the yielded model, as well as learned preference directions.

## Description of the code

The code contain mainly 2 parts : 
   * the first component is about the generation and learning of one parameterized MRSort model (one running of the learning algorithm followed by tests),
   * the second component is about the compilation of series of parameterized MRSort runnings and the output of interesting statistic plots.

### The first component

First, in order to load what is needed in this part, let us excute the following command line :  

In [3]:
run oso-pymcda/apps/random_model_generation_msjp.py

Second, we progressively follow  these steps : 
   * <u>Step 1</u> : initialize both problem and algorithm parameters,
   * <u>Step 2</u> : generate a new random MR-Sort model (profile, weights, threshold) => this is the ground truth model,
   * <u>Step 3</u> : generate randomly a set of alternatives and performance table,
   * <u>Step 4</u> : assign categories to these alternatives to yield a learning set in accordance with the problem addressed,
   * <u>Step 5</u> : run the MR-Sort the readapted metaheuristic algorithm,
   * <u>Step 6</u> : validate the learning of the model (% of classification accuracy (CA) of the learned model compared to the initial model on the learning set, restoration rate of preference directions )
   * <u>Step 7</u> : test the learned algorithm on a benchmarch of alternatives examples
   * <u>Step 8</u> : display the important results (summarized also in a csv file)
   
   

#### Step 1 : initialize the required parameters

Here, we initialize the parameters for one running of the learning algorithm. 
First, we have the problem parameters : 
   * *nb_categories* : the number of categories (classes)
   * *nb_criteria* : the number of criteria taken in consideration in the problem
   * *nb_alternatives* : the number of alternatives in the learning set
   * *dir_criteria* : the list of preference directions of the original model
   * *nb_unk_criteria* : the number of criteria with unknown preference directions
   * *l_dupl_criteria* : the list of criteria (indices) with unknown preference directions
   * *nb_tests* : the number of alternatives taken into account in the test set
   * *nb_models* : the number of models that independantly learn during one running of the learning algorithm
   * *meta_l* : the number of iteration of the metaheuristic algorithm (outer loop)
   * *meta_ll* : the number of iteration of the metaheuristic algorithm (inner loop)
   * *meta_nb_models* : the number of models (population) handled by the metaheuristic (evolutionary) algorithm during the learning process

Let's notice that *nb_unk_criteria* must be smaller than *nb_criteria*.
By default the criteria whose preference directions are known, have an increasing preference direction.
By default the criteria whose preference directions are unknown, are the *nb_unk_criteria* first criteria (the list of criteria starting with c1, c2, c3, ....).
Then, we have the algorithm specific parameters:

   * *version_meta* : the version of implementation
   * *renewal_method* : the method used to renew the population depending on preference directions distribution.  
   * *renewal_models* : the first element of the tuple is the renewal rate, and the second is the coefficient rate (must not be both null or both non null)
   * *strategy* : the first element of the tuple is the starting lower bound on weights, and the second is the starting percentile for the profile interval restriction (on criteria with unknown preference directions)
   * *stopping_condition* : it corresponds to the maximal number of iterations
   * *decision_rule* : it corresponds to the rank of the chosen model among learned models of the population (sorted according to their fitness).
   


In [4]:
nb_categories = 2 # fixed
nb_criteria = 5
nb_alternatives = 50
dir_criteria = [1]*nb_criteria # fixed to 1 for all criteria
nb_unk_criteria = 1
l_dupl_criteria = list(range(nb_criteria))[:nb_unk_criteria]

# test parameters
nb_tests = 10000
nb_models = 5

# parameters of the metaheuristic MRSort
meta_l = 30
meta_ll = 20
meta_nb_models = 50

# additionnal parameters of the algorithm
version_meta = 8 #fixed
renewal_method = 2 #fixed
renewal_models = (0,0.35)
strategy = (0.2,25)
stopping_condition = meta_l
decision_rule = 1


Now we can create an instance of the one running of the learning algorithm as follows :

In [5]:
inst = RandMRSortLearning(nb_alternatives, nb_categories, nb_criteria, dir_criteria, l_dupl_criteria, 
                          nb_tests, nb_models, meta_l, meta_ll, meta_nb_models,renewal_method = renewal_method,
                          renewal_models = renewal_models, strategy = strategy,stopping_condition = stopping_condition, 
                          decision_rule = decision_rule)

#### Step 2 to 4 : generate a new random MRSort model, alternatives and assignments

Here 3 steps are performed one after the other in the same function. We generate a new random MRSort, then we generate alternatives, and finally we assign these alternatives in 2 categories regarding the MRSort rule on the given model. In addition to these 3 operations.

In [6]:
inst.generate_random_instance()

We can have a look on the model that have been generated :
   * generated parameters of the model MRSort

In [7]:
inst.model.bpt.display() # display the limit profile of the random model b1

      c1    c2    c3    c4    c5 
b1 0.491 0.781 0.472 0.303 0.599 


In [8]:
inst.model.cv.display() # display the weights of each criteria of the model w

     c1    c2    c3    c4    c5 
w  0.14 0.085  0.08 0.448 0.247 


In [9]:
print("Majority threshold (lambda) : \t%.7s" % inst.model.lbda) 

Majority threshold (lambda) : 	0.672


   * performance table of generated alternatives

In [10]:
inst.pt.display()

       c1    c2    c3    c4    c5 
a1  0.396 0.795 0.084   0.3 0.433 
a10 0.917 0.175 0.791 0.248 0.011 
a11 0.709 0.993 0.598 0.749 0.887 
a12 0.451 0.204 0.188 0.601 0.643 
a13 0.966 0.873 0.407 0.698 0.109 
a14 0.093 0.005 0.348 0.629 0.173 
a15 0.002 0.962 0.063 0.649 0.942 
a16 0.071   0.3 0.681 0.825 0.665 
a17 0.557 0.939 0.438 0.359 0.856 
a18  0.97 0.832 0.272 0.661  0.54 
a19 0.138 0.173  0.18  0.35 0.413 
a2  0.762 0.897  0.33 0.671 0.086 
a20 0.893 0.313 0.013 0.764 0.785 
a21 0.655 0.613 0.312 0.812 0.889 
a22 0.883 0.309 0.454 0.351 0.421 
a23 0.676 0.885 0.281 0.215  0.03 
a24 0.465 0.341 0.619 0.527 0.975 
a25 0.042 0.232 0.315 0.616 0.291 
a26 0.328 0.579 0.167 0.847 0.162 
a27 0.149  0.12 0.726 0.599 0.521 
a28 0.861 0.663 0.337  0.76 0.656 
a29 0.552 0.976 0.761 0.723 0.659 
a3  0.542 0.548 0.363 0.931 0.967 
a30 0.874 0.713 0.453 0.462 0.689 
a31 0.697 0.922 0.757 0.252 0.257 
a32  0.04 0.557 0.565 0.927 0.403 
a33 0.251 0.677  0.86 0.149 0.755 
a34 0.105 0.163 0.92

   * the result of the assignment of alternatives

In [11]:
inst.aa.display()

    category
a1      cat1
a10     cat1
a11     cat2
a12     cat2
a13     cat2
a14     cat1
a15     cat2
a16     cat2
a17     cat2
a18     cat2
a19     cat1
a2      cat2
a20     cat2
a21     cat2
a22     cat1
a23     cat1
a24     cat2
a25     cat1
a26     cat1
a27     cat1
a28     cat2
a29     cat2
a3      cat2
a30     cat2
a31     cat1
a32     cat1
a33     cat1
a34     cat1
a35     cat2
a36     cat1
a37     cat1
a38     cat1
a39     cat2
a4      cat2
a40     cat1
a41     cat2
a42     cat1
a43     cat1
a44     cat1
a45     cat1
a46     cat1
a47     cat2
a48     cat1
a49     cat1
a5      cat2
a50     cat1
a6      cat2
a7      cat2
a8      cat1
a9      cat1


#### Step 5: run the MRSort metaheuristic learning algorithm

This following step represents one running of the metaheuristic algorithm. This execution learns a randomized model from a generated learning set (performance table and assignments of alternatives).

In [12]:
inst.num_model = 0 # the number of the current running
execution_time = inst.run_mrsort()
print("Time (s) : %f" % execution_time) # computational time of the running

Time (s) : 12.393053


We display the parameters of the model learned :

In [13]:
inst.model2.bpt.display()
inst.model2.cv.display()
print("lambda\t%.7s" % inst.model2.lbda) 

        c1    c2    c3    c4      c5 
b1 0.69701 0.832  0.33 0.336 0.59101 
      c1     c2    c3     c4     c5 
w 0.0001 0.4998     0 0.0002 0.4999 
lambda	0.5001


We also display the learned preference directions : (+) for an increasing direction and (-) for a decreasing preference direction

In [14]:
print(list(inst.model2.criteria))

[c1 (+), c2 (+), c3 (+), c4 (+), c5 (+)]


#### Step 6 : validate the learning of the random model

We can calculate the CA for the validation of the model regarding the learning set.

In [15]:
ca_v,cag_v = inst.eval_model_validation() # calculating the validation rate
print("validation rate : %f" % ca_v)

validation rate : 1.000000


We can also draw the confusion matrix of the validation phase :

In [16]:
matrix = compute_confusion_matrix(inst.aa, inst.aa_learned, inst.model.categories) # construction of the confusion matrix
print_confusion_matrix(matrix, inst.model.categories) # printing the confusion matrix

     cat1 cat2 
cat1   27    0 
cat2    0   23 


#### Step 7 : test the learned algorithm on a benchmarch of alternatives examples

Analogously, we can calculate the CA for the test phase regarding a test set.

In [17]:
ao_tests,al_tests,ca_t,cag_t = inst.eval_model_test()
print("test rate : %f" % ca_t)

test rate : 0.940400


#### Step 8 : show the important results

In order to present generalized statistics, we need to carry out the algorithm runnings several times yielding *nb_models* learned models. To do so, we can straightforwardly execute :  

In [18]:
inst.run_mrsort_all_models()

In [19]:
DATADIR

'/Users/pegdwendeminoungou/python_workspace/MRSort-jupyter-notebook'

As a result, all the tests are done and we have also generated a csv file summarizing the tests and giving details on each one. This file is found on the directory *rand_valid_test_na100_nca2_ncr5-0_dupl1* visible from the root directory of this notebook. The file name begins with "valid_test_dupl...." .

Another csv file is the file that contains more compact data facilitating the drawing of different plots. This file is generated with the command line :

In [20]:
inst.report_plot_results_csv()

It yields a csv file, which name begins by "plot_results...." in the same directory as the previous file.

The final function of this section is the function that ouputs an instance of the learning algorithm (criteria, categories, performance tables and assignments, all codified in a customized syntax)

In [21]:
inst.build_osomcda_instance_random()

'/Users/pegdwendeminoungou/python_workspace/MRSort-jupyter-notebook/rand_valid_test_na50_nca2_ncr5-0_dupl1//osomcda_rand-50-2-5-1-20200427-221950.csv'

This output file is also in the previous directory as the previous files.

In [22]:
print("CA (validation) : " ,inst.stats_cav)
print("CA (generalization) : " ,inst.stats_cag)
print("CA (preference direction) : " ,inst.stats_capd)
print("Time execution (seconds) : " ,inst.stats_time)

CA (validation) :  1.0
CA (generalization) :  0.8812200000000001
CA (preference direction) :  0.8
Time execution (seconds) :  12.200940465927124
